## What are exceptions?

We have noticed several times that when running programs we have encountered errors that have caused the program to terminate. This happens, for example, when we try to access a non-existent element of a list:

In [ ]:
names = ['Otto', 'Hugo', 'Maria']
names[3]
print('This is at the end of the program')

Or when we try to divide a number by 0:


In [ ]:
user_input = 0 
2335 / user_input
print('This is at the end of the program')

Or when we try to open a file that does not exist:

In [ ]:
with open('BLA.txt') as fh:
    print(fh.read())
print('This is at the end of the program')  

Since the output of the `print()` function (line 3 in each case) never occurs, we see that in all cases Python immediately terminates the program as soon as the error occurs. However, this only happens if we do not include a mechanism in our program to react to the exception (i.e., the error). For now, though, all that matters is that there are different types of exceptions. This is good because it allows us to selectively react to certain exceptions in a certain way. We will see how this works later. But first, let's look at a few exception types.

## Exception types

 Here again are the three fragments we used to trigger errors at the beginning of this notebook:

In [ ]:
names = ['Otto', 'Hugo', 'Maria']
names[3]
print('This is at the end of the program')

In [ ]:
user_input = 0 
2335 / user_input
print('This is at the end of the program')

In [ ]:
with open('BLA.txt') as fh:
    print(fh.read())
print('This is at the end of the program')  

If we look closely, we see that there are three different types of errors:

* IndexError at (`names[3]`)
* ZeroDivisionError at (`2335 / user_input`)
* FileNotFoundError at (`open('BLA.txt')`)

So, depending on what kind of error occurred, Python generates a corresponding exception object. 

These errors are organized hierarchically, meaning that each additional level signifies a more specific type of error. 

The complete hierarchy of built-in exceptions can be found here: https://docs.python.org/3/library/exceptions.html#exception-hierarchy. 

So the most general kind of an error is the `BaseException`. Derived from it there is e.g. the `KeyboardInterrupt` exception (thrown when the user aborts the program with `CTRL-C`) or the `Exception`, which itself is base class for many other exceptions, like the `ArithmethicError` or the `LookupError`. The `ArithmeticError` in turn is base class for `FloatingPointError`, `OverflowError` and the just occurred `ZeroDivisionError`.

This principle should be familiar to you. It is inheritance in the sense of object orientation.
Starting from a base class `BaseException` is specialized. I.e. `Exception`is a special case of `BaseException`, `ArithmeticError` again a special case of `Exception` and so on.

If we only draw this section of exceptions as a tree, it looks like this:

![Exception Hierarchy](img/exceptions.png)

The clever thing about inheritance is that the following statements are all correct from Python's point of view:

* A `ZeroDivisionError` is a `ZeroDivisionError`.
* A `ZeroDivisionError` is an `ArithmeticError`
* A `ZeroDivisionError` is an `Exception`.
* A `ZeroDivisionError` is a `BaseException`.

So if we want to react to a `ZeroDivisionError`, we can catch exactly this exception type. Alternatively, we could handle all `ArithmeticError` together in one place or even all `Exception` objects, but this is not a good idea.

## Respond to exceptions

So when Python encounters a problem, it creates an exception object. This object can be "caught". Let's look at this with a concrete example:

In [ ]:
divisor = int(input('Divisor: '))
try:
    print(6543 / divisor)
except:
    print('there was an error with the division')
print("the program will be ended.")   

Here we have put the "dangerous" code part (i.e. the part where a division by 0 could happen) into a "`try`" block. If an error occurs within the `try` block, the "`catch`" block is activated. block is activated and (in our trivial example) the corresponding error message is printed there. The important thing here is that -- unlike before -- the program is no longer aborted when the exception occurs, which we can see from the fact that the `print()` function is executed in line 6. We have handled the exception ourselves and Python therefore no longer sees any reason to terminate the program because of the error.

However, what we have done here is **very bad style**: We caught any kind of error or warning. This may also catch errors that we didn't intend to catch, and which could possibly give crucial hints for troubleshooting. Here is a (very contrived) example to illustrate this. First enter `0` to trigger a `ZeroDivisionError`:

In [ ]:
divisor = int(input('Divisor: '))
some_names = ['Otto', 'Anna']
try:
    print(some_names[divisor])
    print(6543 / divisor)
except:
    print('An error occurred during division')
print("the program will end normally.")       

This is what we already know and also expected.

Now run the cell above again and enter `5`! Python again reports that there was a problem with the division. In truth, there was nothing wrong with the division; the problem was that in line 4 we accessed index `5` of a list with only two elements. So we are no longer dealing with a `ZeroDivisionError` here, but with an `IndexError`. But since we catch all exceptions in our `catch`, we never get to see the real error message and the debugging will be correspondingly difficult.

That is the reason why a

```python
catch:
```

should never be used in this most general form. Instead, specify the type of exception to be responded to as precisely as possible (line 6):

In [ ]:
divisor = int(input('Divisor: '))
some_names = ['Otto', 'Anna']
try:
    print(some_names[divisor])
    print(6543 / divisor)
except ZeroDivisionError:
    print('An error occurred during division')
print("Program will end normally.")       

The way `catch` is written now, it feels responsible only for exceptions of type `ZeroDivisionError`. Try it out by first typing `0` again and then `5`. At `5` you will now see the `IndexError` again and you have a chance to find out what went wrong.

### Responding to multiple exceptions

In the last example, we raised a `ZeroDivisionError` or an `IndexError` depending on the input. However, only the `ZeroDivisionError` was caught. If we also want to react to the `IndexError` itself, we can add another `catch`:

In [ ]:
divisor = int(input('Divisor: '))
some_names = ['Otto', 'Anna']
try:
    print(some_names[divisor])
    print(6543 / divisor)
except ZeroDivisionError:
    print('An error occurred during division')
except IndexError:
    print('The list doesn\'t have so many elements!')
print("Program will end normally.")   

The art of exception handling is to find out
* when it makes sense to catch an exception at all.
  * If an error is so serious that further program execution makes no sense, it usually makes no sense to catch the exception (unless you want to e.g. issue a user-friendly error message and then terminate the program itself).
  * If the problem is not serious, you could issue a warning in the `catch` block, for example, and then let the program continue to run.
  * Sometimes you may want to keep the program running even in case of serious errors. For example, a web server should not be terminated by an incorrect request. In such a case you catch the exception and send an appropriate request back to the client and then accept the next request.
  * If an error occurs due to immediate user input, it may be possible to request the input again in the `catch` block (but here it is usually better to validate the input explicitly).
  * Sometimes it can make sense to reset the program, e.g. to use a default value. However, this must be well considered.
* At which point the exception should be handled
  * As we will see, the exception does not necessarily have to be caught where it occurs.

Before we go any further, one more example where exception handling might make more sense:

In [ ]:

def ask_for_number():
    """Ask for number.
    
    Ask again if user did enter a non number until he enters a number.
    """
    while True:
        try: 
            num = int(input('give a number: '))
            break
        except ValueError:
            print('this wasn\'t a number!')
    return num
    
print(f'User inputted {ask_for_number()}.')
    

The trick here are the two lines in the ``try`` block: We first try to convert the entered string into an integer. If that works, the next line (``break``) is executed, ending the loop. The converted value is returned. But if something goes wrong during the conversion, Python throws a `ValueError`, which we catch with ``catch`` and output a corresponding message. Since there is no ``break`` in the ``catch`` block, and the ``break`` in the ``try`` block is not reached because of the exception, the user will be prompted again on the next iteration of the loop.

### Use the type hierarchy

Let's get the penultimate example again in a somewhat simplified form:

In [ ]:
divisor = int(input('Divisor: '))
try:
    print(6543 / divisor)
except ZeroDivisionError:
    print('there was an error with the division')
print("the program will end normally.")       

Here we have defined a `catch` block for the `ZeroDivisionError`. Alternatively, we could have gone higher in the exception hierarchy: If we catch all `ArithmeticError`, then the `ZeroDivisionError` is also covered. Enter `0` when running the following code cell:

In [ ]:
divisor = int(input('Divisor: '))
try:
     print(6543 / divisor)
except ArithmeticError:
     print('there was an error with the division')
print("the program will end normally.")

In principle, this would also work:

In [ ]:
divisor = int(input('Divisor: '))

try:
     print(6543 / divisor)
except Exception:
     print('there was an error with the division')
print("the program will end normally.")

However, here we again have a similar problem as at the very beginning when we did not specify an exception type in `catch`: When we catch `exception`, we are responding to the majority of Python error types, making our debugging lives unnecessarily difficult.

On the other hand, if we enter a non-number (e.g. 'abc'), the program is still aborted because the conversion to int is outside the ``try`` block.

<div class="alert alert-block alert-info">
<b>Exercise Exception-1</b>
<p>Run the code cell immediately below and instead of a number, enter the string 'abc'. The program is aborted with an error message. Rewrite the code so that this error is also handled cleanly in the program and the program is thus terminated cleanly.</p>
</div>

In [ ]:
divisor = int(input('Enter a divisor '))
try:
    result = 8374949 / divisor
    print('Result: {}'.format(result))
except (ZeroDivisionError):
    print('there was an eroor with the division.')
print("the program will end normally.")

### finally

If we want to execute a code fragment in any case, i.e. regardless of whether an error occurred or not, we can define a `finally` block. This makes sense for example if we want to release any resources like filehandles.

~~~
try:
    f = open('data.txt')
    # make some computations on data
except ZeroDivisionError:
    print('Warning: Division by Zero in data.txt')
finally:
    f.close()
    
~~~

## Exceptions travel through the stack

A big advantage of exceptions is that we don't necessarily have to catch them where they occur, because they are passed through the program hierarchy until they are handled somewhere (or not, which then leads to program termination). To illustrate, let's write a function that asks the user for a number. Here we must consider that the user could enter something that is not interpretable as a number. In this case, calling the function `int()` (line 3) would raise a `ValueError`:

In [ ]:
def ask_for_int(msg):
    try:
        return int(input('{}: '.format(msg)))
    except ValueError:
        print('this wasn\t a number!')
        return ask_for_int(msg) # Run function again


print(6543 / ask_for_int('give in a divisor'))

We have caught the exception on the spot (namely in the function). Just by the way: in the `catch` block, we call out of the function, the function again until there is a usable input. This is called recursion.

However, since exceptions are passed through the stack, we could catch the exception outside the function (e.g., at a central location in the progrmam). Here is a corresponding example:,

In [ ]:
def ask_for_int(msg):
    return int(input('{}: '.format(msg)))

try:
    print(6543 / ask_for_int('Enter a divisor'))
except ValueError:
    print('this was not a number!')

Small digression: If you want to have a new output here (like in the other example) you can solve this with a `while` loop:

In [ ]:
def ask_for_int(msg):
    return int(input('{}: '.format(msg)))

while True:
    try:
        print(6543 / ask_for_int('Enter a divisor'))
        break
    except ValueError:
        print('this was not a number!')

<b>Digression</b> : The while loop will run as long as the conditional expression evaluates to `True`.

Since `True` always evaluates to `True`, the loop will run indefinitely, until something within the loop returns or breaks.



## Accessing exception objects
When an exception occurs, Python creates an exception object which, as we have seen, is passed through. If needed, we can even examine this object in more detail or use it further in the `catch` block.

In [ ]:
def ask_for_int(msg):
    return int(input('{}: '.format(msg)))

try:
    print(6543 / ask_for_int('enter divisor'))
except ValueError as exp:
    print('i can only devide numbers!')
    print(f'the problem was: {exp.args}')

Via `as` we have stored the exception object in a variable `exp` and use this in line 8 to extract and output the original error message.

## Throw exceptions
We can even throw exceptions ourselves if needed.

In [ ]:
def ask_for_int(msg):
    return int(input('{}: '.format(msg)))

try:
    print(6543 / ask_for_int('enter a number: '))
except ValueError as exp:
    print('there was an error: inavlid input')
    raise exp

Here, as before, we have stored the Exception object in a variable. In the `catch` block we first printed a message and then re-throw the original error. This can be useful if we need to do something on the spot, but the actual exception handling takes place elsewhere.

Experience has shown that you need the option of throwing an exception yourself more often:



In [ ]:
def ask_for_int(msg):
    return int(input('{}: '.format(msg)))

grade = ask_for_int('enter a grade: ')
if grade < 1 or grade > 5:
    raise ValueError("Input must be 1, 2, 3, 4 or 5!")

## Define your own exceptions

In larger projects it is sometimes useful to define your own exceptions or even entire exception hierarchies in order to be able to react specifically to such exceptions.

In [ ]:
class MyAppException(Exception): pass
class MyAppWarning(MyAppException): pass
class MyAppError(MyAppException): pass
class GradeValueException(MyAppError): pass

With that, we have defined our own exception hierarchy:

~~~
exception
|---- MyAppException
       | ---- MyAppWarning
       | ---- MyAppError
              | ---- GradeValueException
~~~

Depending on what is needed, we can react here, for example, to all subtypes of `MyAppException` or to all `MyAppWarnings` or `MyAppErrors` or specifically to a `GradeValueException`.

## In-depth literature
I highly recommend reading at least one of the following resources for more in-depth knowledge!

   * PythonTutorials:
* Chapter 8
* http://docs.python.org/3/tutorial/errors.html
   * Small, Course:
      (https://www.diveinto.org/python3/your-first-python-program.html#exceptions)
   * Pilgrim: Chapter 1.7:
     (https://www.diveinto.org/python3/your-first-python-program.html#exceptions)